## 模型部署与模型监控  

模型训练通常在本地环境中进行，训练完成后需要选择最优模型部署到生产环境。  

模型上线后，客群的变化可能导致模型效果衰减；或者，随着数据维度的增加，模型效果进一步提升；亦或，由于业务调整，某些数据维度无法再使用。  

模型迭代优化可以使得模型在近期样本上表现得更好，可以从模型融合、建模时效和拒绝推断3个角度进行。  


## 1. 模型部署  

### 基于Python开发的模型，部署到线上生产环境时，需要考虑跨语言部署。模型文件可以选择Pickle或者PMML格式。

In [2]:
# Pickle方式保存和读取模型
def save_model_as_pkl(model, path):
    """
    保存模型到路径path
    :param model: 训练完成的模型
    :param path: 保存的目标路径
    """
    import pickle
    with open(path, 'wb') as f:
        # protocol=2，表示以新二进制协议对模型内容进行序列化存储
        # 用于解决Python3环境中保存的模型在Python2环境中读取的问题
        pickle.dump(model, f, protocol=2)


def load_model_from_pkl(path):
    """
    从路径path加载模型
    :param path: 保存的目标路径
    """
    import pickle
    with open(path, 'rb') as f:
        model = pickle.load(f)
    return model


In [5]:
# PMML方式保存和读取模型
# PMML是一套与平台和环境无关的模型表示语言，可实现跨平台的机器学习模型部署
# 因为sklearn2pmml需要Java运行环境，notebook环境暂不支持
from sklearn2pmml import sklearn2pmml, PMMLPipeline
from sklearn_pandas import DataFrameMapper
from pypmml import Model
from xgboost.sklearn import XGBClassifier

# 以xgb模型为例，方式1：
# sklearn接口的xgboost，可使用sklearn2pmml生成pmml文件
def save_model_as_pmml(x, y, save_file_path):
    """
    保存模型到路径save_file_path
    :param x: 训练数据特征
    :param y: 训练数据标签
    :param save_file_path: 保存的目标路径
    """
    # 设置pmml的pipeline
    xgb = XGBClassifier(random_state=88)
    mapper = DataFrameMapper([([i], None) for i in x.columns])
    pipeline = PMMLPipeline([('mapper', mapper), ('classifier', xgb)])
    # 模型训练
    pipeline.fit(x, y)
    # 模型结果保存
    sklearn2pmml(pipeline, save_file_path, with_repr=True)


# PMML格式读取
def load_model_from_pmml(load_file_path):
    """
    从路径load_file_path加载模型
    :param load_file_path: pmml文件路径
    """
    model = Model.fromFile(load_file_path)
    return model


In [6]:
from utils import data_utils

train_x, test_x, train_y, test_y = data_utils.get_x_y_split(test_rate=0.2)
save_model_as_pmml(train_x, train_y, 'data/xgb_model.pmml')
model = load_model_from_pmml('data/xgb_model.pmml')
pre = model.predict(test_x)
print(pre.head())

In [ ]:

# 方式2：
# 原生xgboost.core库生成的XGBoost模型，不能使用sklearn2pmml生成pmml文件，只能通过jpmml-xgboost包，将已有的.bin或.model
# 格式模型文件转为pmml文件

# step1.获取到xgb模型文件
xgb_model = load_model_from_pkl("data/model/xgb_model.pkl")


# step2.生成fmap文件
def create_feature_map(file_name, features):
    outfile = open(file_name, 'w')
    for i, feat in enumerate(features):
        outfile.write('{0}\t{1}\tq\n'.format(i, feat))


create_feature_map('data/model/xgb_model.fmap', xgb_model.feature_names)

# step3.jpmml-xgboost的环境配置及pmml转换：
# step3.1. 下载jpmml-xgboost
# step3.2. 命令行切换到jpmml-xgboost的项目文件夹，输入代码编译
# mvn clean install
# 该步执行完后，jpmml-xgboost的项目文件夹下会多出一个target文件夹，里面包含生成好的jar包
# step3.3. jar包转换为pmml文件
# java -jar jpmml-xgboost_path/target/jpmml-xgboost-executable-1.5-SNAPSHOT.jar  --X-nan-as-missing False
# --model-input data/model/xgb.model --fmap-input data/model/xgb.fmap --target-name target
# --pmml-output data/model/xgb_pmml.pmml

## 2. 模型监控  

模型上线后，为了保证模型有效运行，需要对模型相关指标进行监控。  

1）准确性：定期拉取线上模型打分用到的原始数据，线下重新计算特征并生成模型分，对比线上线下模型分数的一致性。  
2）稳定性：特征的PSI。  
3）有效性：AUC和KS指标，以及主要特征的IV指标

## 3. 模型融合  

将多个模型的结果进行组合，以提升最终模型效果。

## 4. 拒绝推断  

贷前风险预测模型的使用场景是所有授信申请客户，包含被拒绝样本。也就是说，训练模型时使用的客群仅仅是预测人群中的一部分，存在“样本偏差”问题。  

样本偏差会带来模型参数估计的偏差，导致模型效果衰减。如果能在建模样本中加入被拒绝的样本，那么模型的效果可以得到保障。  

问题在于，被拒绝的样本没有标签，而推测被拒绝样本的标签就是“拒绝推断”研究的主要内容。